# Segmenting and Clustering Neighborhoods in Toronto

## Install statements

In [48]:
!pip install geopy
!pip install folium

## Import Statements

In [49]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

### Creating Dataframe

In [50]:
toronto_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Question 1

### Ignoring cells with Borough as Not assigned. Assumed that Not assigned was spelled out (and not just NA or N/A).

In [51]:
toronto_df.drop(toronto_df[toronto_df['Borough'].str.lower() =='not assigned'].index,inplace=True)
toronto_df.reset_index(drop=True, inplace=True)

### Replacing Neighbourhoods that are Not Assigned with Borough. Assumed that Not assigned was spelled out (and not just NA or N/A).

In [52]:
toronto_df['Neighbourhood'] = np.select([toronto_df['Neighbourhood'].str.lower() == 'not assigned'],[toronto_df['Borough']],default=toronto_df['Neighbourhood'])
toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Getting shape of filtered dataframe

In [53]:
rows,columns = toronto_df.shape
print("There are {} rows and {} columns".format(rows,columns))

There are 103 rows and 3 columns


## Question 2

### Getting the postal codes' longitudes from the csv file.

In [54]:
postal_df = pd.read_csv('http://cocl.us/Geospatial_data')
postal_df.head()##shows that only columns are Postal Code, Latitude, and Longitude

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### We now will merge the two data frames of toronto_df and postal_df by Postal Code. We ensure that only rows in toronto_df remain and no new rows get added.

In [55]:
toronto_df = pd.merge(toronto_df,postal_df,on="Postal Code")##this ensures that we are only including data where we have postal code in both data frames.
rows,columns = toronto_df.shape
print("There are {} rows and {} columns".format(rows,columns))
toronto_df.head()

There are 103 rows and 5 columns


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Question 3

### We will now attempt to cluster the Toronto data based on location.

In [56]:
k_clusters = 5

### We get clustering by only considering the location (so no chance other attributes will impact K Means)

In [57]:
torontoKMeans = KMeans(n_clusters=k_clusters,random_state=0).fit(toronto_df[['Latitude','Longitude']])
clusterLabels = torontoKMeans.labels_
toronto_df_with_clusters = toronto_df.copy()
toronto_df_with_clusters["Cluster Label"] = clusterLabels

### I will now follow the steps from previous labs to draw map with labeled clusters on it. First I will initialize with a latitude that is centroid of all relevant data points

In [58]:
centeredLatitude = toronto_df_with_clusters["Latitude"].mean()
centeredLongitude = toronto_df_with_clusters["Longitude"].mean()

### Then I will initialize a folium map

In [68]:
toronto_map_clusters = folium.Map(location=[centeredLatitude, centeredLongitude], zoom_start=11)

### Then I will set color scheme for clusters as seen in previous lab

In [69]:
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

### Then I will Add Markers to the Map

In [70]:
for latitude,longitude,neighbourhood,cluster_label in toronto_df_with_clusters[["Latitude","Longitude","Neighbourhood","Cluster Label"]].to_records(index=False):
                                                                               ##zip(toronto_df_with_clusters["Latitude"],toronto_df_with_clusters["Longitude"])
# for toronto_row in toronto_df_with_clusters:
#     print(toronto_row)
#     latitude,longitude,neighbourhood,cluster_label = toronto_row.loc[['Latitude','Longitude','Neighbourhood','Cluster Label']]
    label = folium.Popup(str(neighbourhood) + ' Cluster ' + str(cluster_label), parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color=rainbow[cluster_label-1],
        fill=True,
        fill_color=rainbow[cluster_label-1],
        fill_opacity=0.7).add_to(toronto_map_clusters)

### Map of Toronto Neighbourhoods with Clusters displayed

In [71]:
toronto_map_clusters